Cross Validation 1

size =  20247

test size 4k+

epoch-5

apply sigmoid (prediction probabilities are logits)

groups-9

target_list = ['Camera', 'Location', 'Microphone', 'Contacts', 'Storage', 'Phone', 'SMS', 'Call_Log', 'Calendar']

threshold-tuning = yes

df_2k = df[(df['Rating'] >= 4.5) & (df['Maximum_Installs'] >= 20000)]

accuracy score : F1 score (micro) and ROC_AUC score

## 1. Setup

In [1]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [2]:
print(tf.__version__)

2.8.0


In [3]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


## 2. Load Dataset

In [4]:
## import libraries

import itertools
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from sklearn import preprocessing
%matplotlib inline

In [5]:
## uploading csv files on drive (to avoid uploading on colab in every session)

from google.colab import drive
drive.mount("/content/drive/")

## drive path
train_path = "/content/drive/MyDrive/MetadataCSV/data_20247/CV_df_train_1.csv"
val_path = "/content/drive/MyDrive/MetadataCSV/data_20247/CV_df_val_1.csv"
test_path = "/content/drive/MyDrive/MetadataCSV/test_dataset.csv"

Mounted at /content/drive/


In [6]:
df_train = pd.read_csv(train_path) 
df_val = pd.read_csv(val_path)
df_test = pd.read_csv(test_path)

print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(16198, 23)
(4049, 23)
(4624, 23)


In [7]:
df_train.head(2)

,App_Name,App_Id,Category,Rating,Maximum_Installs,Editors_Choice,Description,Privacy_Policy,Sensors,Camera,...,Contacts,SMS,Storage,Phone,Get_Accounts,Call_Log,desc_length,Clean_Description,clean_desc_length,token_length
0,Fortune Teller Khmer,com.khemarasoft.fortuneteller,Lifestyle,4.5,50284,False,Want to find out your future? This app is a mu...,http://khemarasoft.com/app-policy,0,0,...,0,0,0,0,0,0,370,want to find out your future? this app is a mu...,365,76
1,Mrsool | مرسول,com.mrsool,Shopping,4.5,6450386,False,Mrsool is one of the largest delivery platform...,https://s.mrsool.co/privacy_policy.html,0,1,...,0,0,1,1,1,0,1356,mrsool is one of the largest delivery platform...,1310,267


In [8]:
target_list = ['Camera', 'Location', 'Microphone', 'Contacts', 'Storage', 'Phone', 'SMS', 'Call_Log', 'Calendar']

In [9]:
# getting number of nonzeros in each column
df_train[target_list].astype(bool).sum(axis=0)

Camera        2491
Location      2465
Microphone    1491
Contacts      1669
Storage       8223
Phone         2411
SMS             60
Call_Log        35
Calendar       279
dtype: int64

In [10]:
df_val[target_list].astype(bool).sum(axis=0)

Camera         607
Location       615
Microphone     359
Contacts       381
Storage       2030
Phone          588
SMS             11
Call_Log         8
Calendar        73
dtype: int64

In [11]:
df_test[target_list].astype(bool).sum(axis=0)

Camera         745
Location       716
Microphone     436
Contacts       500
Storage       2402
Phone          652
SMS             11
Call_Log         6
Calendar        90
dtype: int64

## 3. Data Preprocess

#### 3.1 Clean Text

In [12]:
import nltk

nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
from nltk.corpus import stopwords

stop = set(stopwords.words("english"))

def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]

    return " ".join(text)

In [14]:
df_train["Clean_Description"] = df_train["Clean_Description"].map(remove_stopwords)
df_val["Clean_Description"] = df_val["Clean_Description"].map(remove_stopwords)
df_test["Clean_Description"] = df_test["Clean_Description"].map(remove_stopwords)

In [15]:
import string

def remove_punct(text):
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)

In [16]:
df_train["Clean_Description"] = df_train["Clean_Description"].map(lambda x: remove_punct(x))
df_val["Clean_Description"] = df_val["Clean_Description"].map(lambda x: remove_punct(x))
df_test["Clean_Description"] = df_test["Clean_Description"].map(lambda x: remove_punct(x))

#### 3.2 Create Corpus

In [17]:
from keras.layers import *

In [18]:
import os
import sys
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Activation, Conv2D, Input, Embedding, Reshape, MaxPool2D, Concatenate, Flatten, Dropout, Dense, Conv1D
from keras.layers import MaxPool1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [19]:
from nltk.tokenize import word_tokenize
import nltk


def create_corpus_tk(df):
    corpus = []
    for text in df["Clean_Description"]:
        words = [word.lower() for word in word_tokenize(text)]
        corpus.append(words)
    return corpus

In [20]:
df_T = df_train.append(df_val)
df_T.reset_index(drop=True,inplace=True)

In [21]:
corpus = create_corpus_tk(df_T)

In [22]:
num_words = len(corpus)
print(num_words)

20247


In [23]:
corpus[0]

['want',
 'find',
 'future',
 'app',
 'must',
 'like',
 'read',
 'fortune',
 'fortune',
 'teller',
 'khmer',
 'collection',
 'khmer',
 'language',
 'simple',
 'way',
 'laughs',
 'fun',
 'simple',
 'use',
 'input',
 'phone',
 'number',
 'name',
 'birthday',
 'etc',
 'application',
 'intended',
 'entertainment',
 'purpose',
 'actually',
 'capable',
 'telling',
 'future']

##4. Train Validation Split

In [ ]:
# split the data into a training set and a validation set

In [24]:
train_inputs = df_train["Clean_Description"]
validation_inputs = df_val["Clean_Description"]
train_labels = df_train[target_list]
validation_labels = df_val[target_list]

## 5. Tokenization

In [25]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_inputs)

In [26]:
MAX_SEQUENCE_LENGTH = 600

In [27]:
train_sequences = tokenizer.texts_to_sequences(train_inputs)

In [28]:
train_inputs

0        want find future app must like read fortune fo...
1        mrsool one largest delivery platforms kingdom ...
2        hombi social app managing apartment buildings ...
3        oos skin playerpro free color accents pink blu...
4        lokseva academy online platform managing data ...
                               ...                        
16193    looking best diet plan weight loss thonon diet...
16194    lifemap tool interactively explore tree life z...
16195    free translator quickly translate german portu...
16196    application dulhan mehndi designs offline cont...
16197    gives us immense pleasure pride announce enter...
Name: Clean_Description, Length: 16198, dtype: object

In [29]:
train_sequences

[[37,
  29,
  475,
  1,
  379,
  19,
  139,
  2313,
  2313,
  7432,
  1094,
  165,
  1094,
  44,
  50,
  70,
  15866,
  130,
  50,
  6,
  338,
  7,
  119,
  265,
  930,
  58,
  11,
  1488,
  996,
  1098,
  2181,
  5453,
  3818,
  475],
 [12910,
  18,
  1886,
  1131,
  1985,
  2692,
  12910,
  175,
  2115,
  84,
  4107,
  1793,
  92,
  2346,
  943,
  751,
  1131,
  39,
  154,
  23,
  2840,
  1,
  4683,
  471,
  478,
  1133,
  3031,
  631,
  1924,
  1556,
  2692,
  2840,
  3318,
  271,
  5704,
  4950,
  3545,
  6395,
  738,
  817,
  2307,
  12910,
  1131,
  1,
  2617,
  1653,
  12910,
  19,
  4418,
  224,
  1455,
  895,
  316,
  2617,
  588,
  3031,
  15,
  2617,
  400,
  261,
  1402,
  4209,
  2443,
  2431,
  137,
  4141,
  947,
  6732,
  847,
  12910,
  544,
  359,
  12910,
  2100,
  4683,
  471,
  124,
  2347,
  37,
  359,
  977,
  6,
  12910,
  4579,
  6197,
  427,
  316,
  606,
  413,
  814,
  1653,
  316,
  87,
  316,
  163,
  1435,
  316,
  1924,
  3031,
  1133,
  11607,
  224,
  

In [30]:
train_padded = pad_sequences(
    train_sequences, maxlen=MAX_SEQUENCE_LENGTH, truncating="post", padding="post"
)

In [31]:
train_padded

array([[   37,    29,   475, ...,     0,     0,     0],
       [12910,    18,  1886, ...,     0,     0,     0],
       [19458,   159,     1, ...,     0,     0,     0],
       ...,
       [    4,   394,   256, ...,     0,     0,     0],
       [   11, 18886,  3048, ...,     0,     0,     0],
       [  369,    27,  7690, ...,     0,     0,     0]], dtype=int32)

In [32]:
validation_sequences = tokenizer.texts_to_sequences(validation_inputs)
validation_padded = pad_sequences(
    validation_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [33]:
validation_padded

array([[1197, 1500,    2, ...,    0,    0,    0],
       [   1,  117, 7314, ...,    0,    0,    0],
       [ 810,  411, 1123, ...,    0,    0,    0],
       ...,
       [3992, 2392,  996, ...,    0,    0,    0],
       [ 213,  787,    3, ...,    0,    0,    0],
       [ 116,  196,  794, ...,    0,    0,    0]], dtype=int32)

In [34]:
print(df_train.Clean_Description[0])
print(train_sequences[0])

want find future app must like read fortune fortune teller khmer collection khmer language simple way laughs fun simple use input phone number name birthday etc application intended entertainment purpose actually capable telling future
[37, 29, 475, 1, 379, 19, 139, 2313, 2313, 7432, 1094, 165, 1094, 44, 50, 70, 15866, 130, 50, 6, 338, 7, 119, 265, 930, 58, 11, 1488, 996, 1098, 2181, 5453, 3818, 475]


In [35]:
word_index = tokenizer.word_index
print("Number of unique words:", len(word_index))

Number of unique words: 66465


In [36]:
word_index

{'app': 1,
 'keyboard': 2,
 'theme': 3,
 'free': 4,
 'wallpaper': 5,
 'use': 6,
 'phone': 7,
 'wallpapers': 8,
 'new': 9,
 'features': 10,
 'application': 11,
 'get': 12,
 'time': 13,
 'make': 14,
 'also': 15,
 'android': 16,
 'launcher': 17,
 'one': 18,
 'like': 19,
 'download': 20,
 'screen': 21,
 'hd': 22,
 'best': 23,
 'english': 24,
 'easy': 25,
 'learn': 26,
 'us': 27,
 'live': 28,
 'find': 29,
 'themes': 30,
 'love': 31,
 'help': 32,
 'share': 33,
 'mobile': 34,
 'please': 35,
 'set': 36,
 'want': 37,
 'support': 38,
 'apps': 39,
 'words': 40,
 'device': 41,
 'home': 42,
 'need': 43,
 'language': 44,
 'day': 45,
 'using': 46,
 'enjoy': 47,
 'available': 48,
 'images': 49,
 'simple': 50,
 'many': 51,
 'different': 52,
 'note': 53,
 'beautiful': 54,
 'add': 55,
 'information': 56,
 'text': 57,
 'etc': 58,
 'access': 59,
 'apply': 60,
 'learning': 61,
 'background': 62,
 'you': 63,
 'life': 64,
 'every': 65,
 'daily': 66,
 'video': 67,
 'friends': 68,
 'install': 69,
 'way': 70,
 '

In [37]:
word_index["reason"]

2506

In [38]:
print(validation_sequences[0])

[1197, 1500, 2, 3, 9, 91, 425, 130, 240, 48, 12, 1197, 1500, 2, 3, 1524, 1260, 14, 7, 158, 1478, 527, 1189, 1197, 1500, 2, 3, 544, 7, 2074, 405, 2032, 3726, 1197, 1500, 2, 3, 1359, 148, 84, 20, 69, 1197, 1500, 2, 3, 4, 132, 88, 84, 2947, 2876, 2470, 72, 199, 1197, 1500, 2, 3, 598, 197, 7, 2121, 2446, 1197, 1500, 2, 3, 2328, 31, 1197, 1500, 2, 3, 6, 1197, 1500, 2, 3, 53, 35, 20, 832, 2, 154, 20, 1197, 1500, 2, 3, 80, 78, 104, 142, 87, 60, 133, 512, 266, 1197, 1500, 2, 3, 837, 339, 504, 1197, 1500, 2, 3, 160, 204, 185, 248, 122, 245, 248, 102, 36, 22, 8, 6, 40, 743, 14, 362, 348, 10, 93, 327, 999, 148, 207, 83, 726, 203, 628, 488, 726, 172, 338, 325, 1305, 537, 1426, 2030, 2, 426, 9, 10, 1204, 148, 3071, 84, 2, 402, 686, 439, 82, 38, 106, 451, 595, 2, 405, 2, 16, 309, 158, 16, 30, 1628, 2, 3, 962, 2, 134, 365, 4, 164, 158, 418, 2, 22, 8, 388, 7, 62, 19, 360, 22, 5, 118, 22, 5, 184, 22, 5, 884, 599, 710, 22, 8, 1378, 30, 91, 30, 118, 30, 658, 30, 1087, 30, 143, 1149, 3, 58, 224, 29, 97, 2

In [39]:
word_index["listen"]

301

## 6. Create the Embedding dictionary

In [40]:
embedding_dict = {}
with open("/content/drive/MyDrive/glove6B/glove.6B.300d.txt", "r") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], "float32")
        embedding_dict[word] = vectors
f.close()

In [41]:
embedding_dict

Output hidden; open in https://colab.research.google.com to view.

In [42]:
embedding_dim = 300

In [43]:
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, embedding_dim))

for word, i in word_index.items():
    if i < num_words:
        emb_vec = embedding_dict.get(word)
        if emb_vec is not None:
            embedding_matrix[i] = emb_vec

In [44]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.73247999, -0.078309  ,  0.018521  , ...,  0.21988   ,
        -0.13121   ,  0.043819  ],
       [-0.51651001,  0.090911  , -0.15025   , ...,  0.17309999,
         0.094735  , -0.62935001],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.074421  , -0.33252001, -0.56362998, ..., -0.057775  ,
         0.17037   , -0.54825002],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [45]:
word_index["reason"]

2506

In [46]:
embedding_dict.get("reason")

array([ 1.3197e-01, -1.2591e-01,  4.3864e-02,  3.6321e-02,  9.6646e-02,
       -1.3829e-01,  3.8637e-01,  7.6962e-02, -1.1306e-01, -1.6083e+00,
        2.0062e-02, -5.2665e-02, -1.6597e-01,  1.2171e-01,  2.8945e-01,
       -1.7289e-01,  5.3035e-02, -2.7842e-01,  8.2376e-02, -1.1980e-02,
        3.7228e-02,  2.1867e-01,  1.5267e-01, -8.4361e-02, -3.1292e-01,
       -3.2093e-02,  2.0281e-01, -3.5910e-01,  1.6873e-02, -2.2996e-01,
        2.6044e-02,  3.5910e-01, -3.2431e-01, -5.4194e-01, -9.7742e-01,
        3.9198e-02, -1.7794e-01,  7.4200e-02, -4.1251e-02, -7.8917e-02,
        2.0646e-01, -6.6538e-02,  6.7401e-02,  1.4965e-01,  5.9107e-02,
       -3.7585e-02, -3.4672e-02,  5.5291e-02, -8.5636e-02,  9.1743e-02,
        4.9125e-01,  7.5606e-03, -3.0860e-01,  5.8902e-04, -8.6975e-02,
        3.9904e-01, -1.2695e-01,  2.2471e-01,  2.3658e-01,  3.0489e-01,
       -6.7363e-02,  3.5839e-01,  4.9703e-01,  4.1895e-01, -3.8494e-01,
       -2.6257e-01,  1.6049e-01, -1.0992e-01,  2.7477e-02,  1.49

In [47]:
(embedding_matrix[2363] == embedding_dict.get("reason")).all()

False

In [48]:
print(train_padded.shape)
print(train_labels.shape)

(16198, 600)
(16198, 9)


In [49]:
print(validation_padded.shape)
print(validation_labels.shape)

(4049, 600)
(4049, 9)


## 7. Build CNN Model

In [50]:
num_permissions = 9 #11
drop = 0.2

batch_size = 32
max_train_epochs = 300
validation_split = 6

early_stopping_patience = 16
early_stopping_delta = 0.02  # 2%

max_description_embeddings = 600
embedding_dim = 300  # +1 for flag

#downloaded_embedding_file = data_folder + "/word_embeddings/glove.6B.300d.txt"


conv_filters_num = 1024
conv_filters_sizes = [1, 2, 3]
dense_layers = [5000, 2500]
dropout = 0.2

heatmap_threshold = 0.49

In [51]:
# Function for class weights

import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import MultiLabelBinarizer


def generate_class_weights(class_series, multi_class=True, one_hot_encoded=False):
  """
  Method to generate class weights given a set of multi-class or multi-label labels, both one-hot-encoded or not.
  Some examples of different formats of class_series and their outputs are:
    - generate_class_weights(['mango', 'lemon', 'banana', 'mango'], multi_class=True, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 1.3333333333333333, 'mango': 0.6666666666666666}
    - generate_class_weights([[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0]], multi_class=True, one_hot_encoded=True)
    {0: 0.6666666666666666, 1: 1.3333333333333333, 2: 1.3333333333333333}
    - generate_class_weights([['mango', 'lemon'], ['mango'], ['lemon', 'banana'], ['lemon']], multi_class=False, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 0.4444444444444444, 'mango': 0.6666666666666666}
    - generate_class_weights([[0, 1, 1], [0, 0, 1], [1, 1, 0], [0, 1, 0]], multi_class=False, one_hot_encoded=True)
    {0: 1.3333333333333333, 1: 0.4444444444444444, 2: 0.6666666666666666}
  The output is a dictionary in the format { class_label: class_weight }. In case the input is one hot encoded, the class_label would be index
  of appareance of the label when the dataset was processed. 
  In multi_class this is np.unique(class_series) and in multi-label np.unique(np.concatenate(class_series)).
  Author: Angel Igareta (angel@igareta.com)
  """
  if multi_class:
    # If class is one hot encoded, transform to categorical labels to use compute_class_weight   
    if one_hot_encoded:
      class_series = np.argmax(class_series, axis=1)
  
    # Compute class weights with sklearn method
    class_labels = np.unique(class_series)
    class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=class_series)

    print(class_series)
    print(class_labels)

    return dict(zip(class_labels, class_weights))
  else:
    # It is neccessary that the multi-label values are one-hot encoded
    mlb = None
    if not one_hot_encoded:
      mlb = MultiLabelBinarizer()
      class_series = mlb.fit_transform(class_series)

    n_samples = len(class_series)
    n_classes = len(class_series[0])
    print(n_samples)
    print(n_classes)

    # Count each class frequency
    class_count = [0] * n_classes
    for classes in class_series:
        for index in range(n_classes):
            if classes[index] != 0:
                class_count[index] += 1
    
    # Compute class weights using balanced method
    class_weights = [n_samples / (n_classes * freq) if freq > 0 else 1 for freq in class_count]
    class_labels = range(len(class_weights)) if mlb is None else mlb.classes_
    return dict(zip(class_labels, class_weights))
    #return class_weights

In [52]:
class_series = np.array(train_labels)
class_wt = generate_class_weights(class_series, multi_class=False, one_hot_encoded=True)
print(class_wt)
#class_wt = torch.tensor(class_wt)
#print(class_wt)

16198
9
{0: 0.7225121548686382, 1: 0.7301329727293216, 2: 1.207094418362024, 3: 1.0783569669129884, 4: 0.21887118786060777, 5: 0.7464860131803309, 6: 29.996296296296297, 7: 51.422222222222224, 8: 6.450816407805655}


In [53]:
def model_multiconv_1d(num_permissions):
    #embedding_dim = embedding_dim
    sequence_length = max_description_embeddings

    input_layer = Input(shape=(None,))

    conv_layers = []
    for filter_size in conv_filters_sizes:
        conv_layer_i = Embedding(num_words,
                                 output_dim=embedding_dim,
                                 input_length=sequence_length,
                                 weights=[embedding_matrix],
                                 trainable=False)(input_layer)
        conv_layer_i = Conv1D(filters=conv_filters_num,
                              kernel_size=filter_size,
                              padding='same',
                              activation='relu')(conv_layer_i)
        conv_layer_i = GlobalMaxPooling1D()(conv_layer_i)

        conv_layers.append(conv_layer_i)

    if len(conv_layers) == 1:
        previous_layer = conv_layers[0]
    else:
        concatenated_layer = concatenate(conv_layers, axis=-1)
        previous_layer = concatenated_layer

    for n_neurons in dense_layers:
        previous_layer = Dense(n_neurons, activation='relu')(previous_layer)
        previous_layer = Dropout(dropout)(previous_layer)

    output_layer = Dense(num_permissions, activation='sigmoid')(previous_layer)

    return keras.Model(inputs=input_layer, outputs=output_layer)

In [54]:
model = model_multiconv_1d(num_permissions)

In [55]:
model.compile(loss="binary_crossentropy", optimizer=Adam(0.0001), metrics=['accuracy'])
              
#metrics=[metrics.fb_micro, metrics.fb_macro, metrics.precision, metrics.recall])
train_metric = 'val_fb_macro'

In [56]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 300)    19939800    ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 300)    19939800    ['input_1[0][0]']                
                                                                                                  
 embedding_2 (Embedding)        (None, None, 300)    19939800    ['input_1[0][0]']                
                                                                                              

In [57]:
checkpoint = ModelCheckpoint('weights_cnn_sentece.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

model.fit(train_padded, train_labels, 
          batch_size=batch_size, epochs=10, class_weight=class_wt,
          verbose=1, callbacks=[checkpoint], validation_data=(validation_padded, validation_labels))

Epoch 1/10
507/507 [==============================] - 112s 199ms/step - loss: 0.1906 - accuracy: 0.2813 - val_loss: 0.2572 - val_accuracy: 0.2959
Epoch 2/10
507/507 [==============================] - 102s 201ms/step - loss: 0.1520 - accuracy: 0.2920 - val_loss: 0.2481 - val_accuracy: 0.3072
Epoch 3/10
507/507 [==============================] - 103s 203ms/step - loss: 0.1346 - accuracy: 0.2965 - val_loss: 0.2122 - val_accuracy: 0.2862
Epoch 4/10
507/507 [==============================] - 101s 199ms/step - loss: 0.1135 - accuracy: 0.2961 - val_loss: 0.2194 - val_accuracy: 0.3319
Epoch 5/10
507/507 [==============================] - 101s 199ms/step - loss: 0.0929 - accuracy: 0.3018 - val_loss: 0.2055 - val_accuracy: 0.3087
Epoch 6/10
507/507 [==============================] - 101s 199ms/step - loss: 0.0807 - accuracy: 0.3108 - val_loss: 0.2404 - val_accuracy: 0.3208
Epoch 7/10
507/507 [==============================] - 102s 200ms/step - loss: 0.0643 - accuracy: 0.3291 - val_loss: 0.2479 -

## 8. Performance on Test Set

In [58]:
test_labels = df_test[target_list]
test_input = df_test["Clean_Description"]

In [59]:
test_sequences = tokenizer.texts_to_sequences(test_input)
test_padded = pad_sequences(
    test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [60]:
print(df_test.Clean_Description[0])
print(test_sequences[0])

poetry lover find best poetry poets urdu problem solved app app contains time best urdu poetry famous urdu poets offline urdu poetry collection best urdu poetry famous urdu poets like mohsin naqvi mirza ghalib saghir siddiqui others also contains romantic urdu poetry sad urdu poetry urdu ghazals offline urdu shayari urdu shayari app contains best ever poetry famous poets times app contains best variety urdu poetry categories romantic poetry sad poetry attitude poetry ghazals inspirational poetry main features offline poetry collection gallery view poetry collection zooming feature save share poetry images app contains offline poetry urdu ghazals urdu poetry poetry famous poets urdu shayari much more
[2156, 1874, 29, 23, 2156, 10817, 401, 729, 1887, 1, 1, 206, 13, 23, 401, 2156, 1168, 401, 10817, 120, 401, 2156, 165, 23, 401, 2156, 1168, 401, 10817, 19, 17769, 428, 15, 206, 462, 401, 2156, 1864, 401, 2156, 401, 8538, 120, 401, 937, 401, 937, 1, 206, 23, 527, 2156, 1168, 10817, 209, 1, 2

In [61]:
print(test_labels)

      Camera  Location  Microphone  Contacts  Storage  Phone  SMS  Call_Log  \
0          0         0           0         0        1      0    0         0   
1          0         0           0         0        1      0    0         0   
2          0         0           0         0        1      0    0         0   
3          0         0           0         0        0      0    0         0   
4          1         0           0         0        0      1    0         0   
...      ...       ...         ...       ...      ...    ...  ...       ...   
4619       0         1           0         0        0      0    0         0   
4620       0         0           0         0        0      0    0         0   
4621       0         0           0         0        1      0    0         0   
4622       0         1           1         0        1      0    0         0   
4623       0         0           0         0        0      0    0         0   

      Calendar  
0            0  
1            0  


In [62]:
print(test_padded.shape)
print(test_labels.shape)

(4624, 600)
(4624, 9)


In [63]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`

predictions = model.predict(test_padded)
print("predictions shape:", predictions.shape)

predictions shape: (4624, 9)


In [64]:
predictions[0:10]

array([[4.15346818e-04, 4.90719394e-04, 3.87188447e-05, 1.27224084e-05,
        1.31210044e-01, 2.12970379e-04, 5.97426333e-06, 2.83702164e-08,
        1.44099213e-05],
       [1.03089435e-04, 4.82018047e-04, 2.06691802e-05, 1.04827181e-04,
        3.06921080e-03, 4.03632177e-04, 2.27285287e-04, 1.36230710e-06,
        2.12600498e-04],
       [4.30876374e-01, 2.17893515e-02, 4.17164378e-02, 1.14059165e-01,
        9.41546857e-01, 7.00828508e-02, 3.66262253e-03, 2.26264965e-04,
        2.37938091e-02],
       [8.22371861e-04, 1.74088962e-02, 2.52721156e-06, 4.18161973e-02,
        4.75296676e-01, 1.45234866e-02, 4.68651182e-04, 1.88245449e-05,
        7.12638546e-04],
       [3.18957418e-02, 2.14547385e-02, 4.07014486e-05, 1.36308009e-02,
        9.19547603e-02, 7.53441229e-02, 2.92897294e-03, 8.84322653e-05,
        2.39821337e-03],
       [5.56310773e-01, 2.66806819e-02, 1.78953502e-02, 3.63557041e-02,
        9.95552242e-01, 2.81771589e-02, 3.70690192e-04, 1.53948240e-05,
        2.1

In [65]:
true_labels = test_labels.to_numpy()
true_labels[0:10]

array([[0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 1, 0, 1, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0]])

In [66]:
np.save("predictions.npy", predictions)
loaded_predictions = np.load("predictions.npy")
print(loaded_predictions)

[[4.15346818e-04 4.90719394e-04 3.87188447e-05 ... 5.97426333e-06
  2.83702164e-08 1.44099213e-05]
 [1.03089435e-04 4.82018047e-04 2.06691802e-05 ... 2.27285287e-04
  1.36230710e-06 2.12600498e-04]
 [4.30876374e-01 2.17893515e-02 4.17164378e-02 ... 3.66262253e-03
  2.26264965e-04 2.37938091e-02]
 ...
 [8.85262847e-01 4.46089469e-02 1.29511759e-01 ... 3.06184008e-03
  1.73577238e-04 2.37297695e-02]
 [5.26602799e-03 9.97775733e-01 9.95322526e-01 ... 3.14694404e-07
  1.33970532e-08 6.71713322e-04]
 [6.26118999e-05 3.44547807e-05 9.13433541e-05 ... 1.32381858e-04
  1.17483148e-06 1.94346343e-04]]


In [67]:
np.save("true_labels.npy", true_labels)
loaded_true_labels = np.load("true_labels.npy")
print(loaded_true_labels)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


## 9. Threshold Calculation

In [68]:
# Import module for data manipulation
import pandas as pd
# Import module for linear algebra
import numpy as np
# Import module for data simulation
from sklearn.datasets import make_classification     # Create a synthetic dataframe
from sklearn.linear_model import LogisticRegression  # Classification model
from sklearn.model_selection import train_test_split # Split the dataframe
from sklearn.metrics import roc_curve                # Calculate the ROC curve
from sklearn.metrics import precision_recall_curve   # Calculate the Precision-Recall curve
from sklearn.metrics import f1_score                 # Calculate the F-score
# Import module for data visualization
from plotnine import *
import plotnine

In [69]:
thresh_f = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
thresh_roc = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [70]:
n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("predictions.npy")
  true_labels = np.load("true_labels.npy")

  # Array for finding the optimal threshold
  thresholds = np.arange(0.0, 1.0, 0.0001)
  fscore = np.zeros(shape=(len(thresholds)))
  print('Length of sequence: {}'.format(len(thresholds)))

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Fit the model
  for index, elem in enumerate(thresholds):
    # Corrected probabilities
    y_pred_prob = (pred > elem).astype('int')
    # Calculate the f-score
    fscore[index] = f1_score(labels, y_pred_prob)

  # Find the optimal threshold
  index = np.argmax(fscore)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  fscoreOpt = round(fscore[index], ndigits = 4)
  thresh_f[i] = thresholdOpt
  print('Best Threshold: {} with F-Score: {}'.format(thresholdOpt, fscoreOpt))

print("-------------------------------------")
print("optimal threshold tuning for f-score")
print(thresh_f)

permission  0
Length of sequence: 10000
Best Threshold: 0.3521 with F-Score: 0.6742
permission  1
Length of sequence: 10000
Best Threshold: 0.3038 with F-Score: 0.5706
permission  2
Length of sequence: 10000
Best Threshold: 0.1395 with F-Score: 0.5736
permission  3
Length of sequence: 10000
Best Threshold: 0.0721 with F-Score: 0.5348
permission  4
Length of sequence: 10000
Best Threshold: 0.2354 with F-Score: 0.803
permission  5
Length of sequence: 10000
Best Threshold: 0.1367 with F-Score: 0.538
permission  6
Length of sequence: 10000
Best Threshold: 0.0295 with F-Score: 0.1364
permission  7
Length of sequence: 10000
Best Threshold: 0.07 with F-Score: 0.2857
permission  8
Length of sequence: 10000
Best Threshold: 0.1696 with F-Score: 0.314
-------------------------------------
optimal threshold tuning for f-score
[0.3521, 0.3038, 0.1395, 0.0721, 0.2354, 0.1367, 0.0295, 0.07, 0.1696]


In [71]:
##for roc curve with g-mean

n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("predictions.npy")
  true_labels = np.load("true_labels.npy")

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Create the ROC curve
  fpr, tpr, thresholds = roc_curve(labels, pred)

  df_fpr_tpr = pd.DataFrame({'FPR':fpr, 'TPR':tpr, 'Threshold':thresholds})

  # Calculate the G-mean
  gmean = np.sqrt(tpr * (1 - fpr))

  # Find the optimal threshold
  index = np.argmax(gmean)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  gmeanOpt = round(gmean[index], ndigits = 4)
  fprOpt = round(fpr[index], ndigits = 4)
  tprOpt = round(tpr[index], ndigits = 4)

  thresh_roc[i] = thresholdOpt
  print('Best Threshold: {} with G-Mean: {}'.format(thresholdOpt, gmeanOpt))
  print('FPR: {}, TPR: {}'.format(fprOpt, tprOpt))

print("-------------------------------------")
print("ROC curve with G-mean threshold tuning")
print(thresh_roc)

permission  0
Best Threshold: 0.09449999779462814 with G-Mean: 0.8506
FPR: 0.139, TPR: 0.8403
permission  1
Best Threshold: 0.01899999938905239 with G-Mean: 0.7694
FPR: 0.2293, TPR: 0.7682
permission  2
Best Threshold: 0.0034000000450760126 with G-Mean: 0.8209
FPR: 0.1769, TPR: 0.8188
permission  3
Best Threshold: 0.019899999722838402 with G-Mean: 0.8092
FPR: 0.1753, TPR: 0.794
permission  4
Best Threshold: 0.4544000029563904 with G-Mean: 0.7852
FPR: 0.2385, TPR: 0.8097
permission  5
Best Threshold: 0.016200000420212746 with G-Mean: 0.7611
FPR: 0.2694, TPR: 0.7929
permission  6
Best Threshold: 0.004900000058114529 with G-Mean: 0.8717
FPR: 0.0713, TPR: 0.8182
permission  7
Best Threshold: 9.999999747378752e-05 with G-Mean: 0.9102
FPR: 0.1715, TPR: 1.0
permission  8
Best Threshold: 0.006000000052154064 with G-Mean: 0.8061
FPR: 0.1877, TPR: 0.8
-------------------------------------
ROC curve with G-mean threshold tuning
[0.0945, 0.019, 0.0034, 0.0199, 0.4544, 0.0162, 0.0049, 1e-04, 0.006]

## 10. Performance Score

In [ ]:
#Fscore micro for different thresholds-

In [72]:
#predictions = np.load("predictions.npy")
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [73]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import f1_score

def f_at_1(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_f[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_f[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_f[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_f[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_f[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_f[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_f[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_f[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_f[8]

   
    acc[0] = f1_score(labels[:, 0], preds_th[:, 0])
    acc[1] = f1_score(labels[:, 1], preds_th[:, 1])
    acc[2] = f1_score(labels[:, 2], preds_th[:, 2])
    acc[3] = f1_score(labels[:, 3], preds_th[:, 3])
    acc[4] = f1_score(labels[:, 4], preds_th[:, 4])
    acc[5] = f1_score(labels[:, 5], preds_th[:, 5])
    acc[6] = f1_score(labels[:, 6], preds_th[:, 6])
    acc[7] = f1_score(labels[:, 7], preds_th[:, 7])
    acc[8] = f1_score(labels[:, 8], preds_th[:, 8])

   #f1_score_micro = metrics.f1_score(targets, outputs, average='micro')

    #print(acc)
    return acc

In [74]:
eval_accuracy = f_at_1(predictions, true_labels)

np.save("F1_CV1_N20k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.
print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")
avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 score: {0:.4f}".format(avg_score))

  Camera    : 0.6742
  Location  : 0.5706
  Microphone: 0.5736
  Contacts  : 0.5348
  Storage   : 0.8030
  Phone     : 0.5380
  SMS       : 0.1364
  Call_Log  : 0.2857
  Calendar  : 0.3140

  Average F1 score: 0.4923


In [ ]:
#Fscore micro for different thresholds-

In [75]:
#predictions = np.load("predictions.npy")
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [76]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import f1_score

def f1micro_accuracy(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_f[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_f[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_f[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_f[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_f[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_f[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_f[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_f[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_f[8]

    acc[0] = f1_score(labels[:, 0], preds_th[:, 0], average='micro')
    acc[1] = f1_score(labels[:, 1], preds_th[:, 1], average='micro')
    acc[2] = f1_score(labels[:, 2], preds_th[:, 2], average='micro')
    acc[3] = f1_score(labels[:, 3], preds_th[:, 3], average='micro')
    acc[4] = f1_score(labels[:, 4], preds_th[:, 4], average='micro')
    acc[5] = f1_score(labels[:, 5], preds_th[:, 5], average='micro')
    acc[6] = f1_score(labels[:, 6], preds_th[:, 6], average='micro')
    acc[7] = f1_score(labels[:, 7], preds_th[:, 7], average='micro')
    acc[8] = f1_score(labels[:, 8], preds_th[:, 8], average='micro')
    
   #f1_score_micro = metrics.f1_score(targets, outputs, average='micro')

    #print(acc)
    return acc

In [77]:
eval_accuracy = f1micro_accuracy(predictions, true_labels)

np.save("F1Mic_CV1_N20k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 (micro) score: {0:.4f}".format(avg_score))

  Camera    : 0.8873
  Location  : 0.8776
  Microphone: 0.9286
  Contacts  : 0.8769
  Storage   : 0.7794
  Phone     : 0.8659
  SMS       : 0.9918
  Call_Log  : 0.9989
  Calendar  : 0.9821

  Average F1 (micro) score: 0.9098


In [ ]:
#roc-auc score for different thresholds-

In [78]:
import numpy as np
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [79]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import roc_auc_score

def roc_auc(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_roc[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_roc[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_roc[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_roc[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_roc[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_roc[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_roc[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_roc[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_roc[8]


    acc[0] = roc_auc_score(labels[:, 0], preds_th[:, 0])
    acc[1] = roc_auc_score(labels[:, 1], preds_th[:, 1])
    acc[2] = roc_auc_score(labels[:, 2], preds_th[:, 2])
    acc[3] = roc_auc_score(labels[:, 3], preds_th[:, 3])
    acc[4] = roc_auc_score(labels[:, 4], preds_th[:, 4])
    acc[5] = roc_auc_score(labels[:, 5], preds_th[:, 5])
    acc[6] = roc_auc_score(labels[:, 6], preds_th[:, 6])
    acc[7] = roc_auc_score(labels[:, 7], preds_th[:, 7])
    acc[8] = roc_auc_score(labels[:, 8], preds_th[:, 8])
   

    #print(acc)
    return acc

In [80]:
#roc-auc score

eval_accuracy = roc_auc(predictions, true_labels)

np.save("ROC_CV1_N20k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average ROC_AUC score: {0:.4f}".format(avg_score))

  Camera    : 0.8507
  Location  : 0.7694
  Microphone: 0.8190
  Contacts  : 0.8093
  Storage   : 0.7856
  Phone     : 0.7605
  SMS       : 0.8282
  Call_Log  : 0.8356
  Calendar  : 0.8062

  Average ROC_AUC score: 0.8072


# ACNET performance

#### a) Data Preparation

In [81]:
## drive path
acnet_path = "/content/drive/MyDrive/MetadataCSV/acnet_dataset_preprocess.csv"

In [82]:
df_acnet = pd.read_csv(acnet_path) 
print(df_acnet.shape)

(1417, 14)


In [83]:
df_acnet = df_acnet.dropna(subset=['Clean_Description'])

In [84]:
df_acnet.head()

,app_id,description,Storage,Contacts,Location,Camera,Microphone,SMS,Call_Log,Phone,Calendar,Settings,Tasks,Clean_Description
0,0,ROOT is REQUIRED for automatic synchronization...,1,0,1,0,0,0,0,0,0,0,0,root is required for automatic synchronization...
1,1,This app delivers short scriptures containing ...,0,0,0,0,0,1,0,0,0,0,0,this app delivers short scriptures containing ...
2,2,This game is surprisingly simple and very addi...,0,0,0,0,0,0,0,0,0,0,0,this game is surprisingly simple and very addi...
3,3,It is an online RPG game based on LBS location...,0,0,1,0,0,0,0,0,0,0,0,it is an online rpg game based on lbs location...
4,4,Christmas is in the air. Get yourself in the h...,0,1,0,0,0,0,0,0,0,1,1,christmas is in the air. get yourself in the h...


In [85]:
df_acnet["Clean_Description"] = df_acnet["Clean_Description"].map(remove_stopwords)

In [86]:
df_acnet["Clean_Description"] = df_acnet["Clean_Description"].map(lambda x: remove_punct(x))

In [87]:
acnet_labels = df_acnet[target_list]
acnet_input = df_acnet["Clean_Description"]

In [88]:
acnet_sequences = tokenizer.texts_to_sequences(acnet_input)
acnet_padded = pad_sequences(
    acnet_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [89]:
print(acnet_padded.shape)
print(acnet_labels.shape)

(1414, 600)
(1414, 9)


#### b) Get Predictions

In [90]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`

predictions = model.predict(acnet_padded)
print("predictions shape:", predictions.shape)

predictions shape: (1414, 9)


In [91]:
acnet_labels = acnet_labels.to_numpy()

In [92]:
np.save("acnet_predictions.npy", predictions)
loaded_predictions = np.load("acnet_predictions.npy")

np.save("acnet_labels.npy", acnet_labels)
loaded_true_labels = np.load("acnet_labels.npy")

#### c) Threshold Calculation

In [93]:
thresh_f = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
thresh_roc = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [94]:
n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("acnet_predictions.npy")
  true_labels = np.load("acnet_labels.npy")

  # Array for finding the optimal threshold
  thresholds = np.arange(0.0, 1.0, 0.0001)
  fscore = np.zeros(shape=(len(thresholds)))
  print('Length of sequence: {}'.format(len(thresholds)))

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Fit the model
  for index, elem in enumerate(thresholds):
    # Corrected probabilities
    y_pred_prob = (pred > elem).astype('int')
    # Calculate the f-score
    fscore[index] = f1_score(labels, y_pred_prob)

  # Find the optimal threshold
  index = np.argmax(fscore)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  fscoreOpt = round(fscore[index], ndigits = 4)
  thresh_f[i] = thresholdOpt
  print('Best Threshold: {} with F-Score: {}'.format(thresholdOpt, fscoreOpt))

print("-------------------------------------")
print("optimal threshold tuning for f-score")
print(thresh_f)

permission  0
Length of sequence: 10000
Best Threshold: 0.2838 with F-Score: 0.4708
permission  1
Length of sequence: 10000
Best Threshold: 0.0993 with F-Score: 0.5665
permission  2
Length of sequence: 10000
Best Threshold: 0.0376 with F-Score: 0.5045
permission  3
Length of sequence: 10000
Best Threshold: 0.0256 with F-Score: 0.6054
permission  4
Length of sequence: 10000
Best Threshold: 0.0278 with F-Score: 0.5941
permission  5
Length of sequence: 10000
Best Threshold: 0.267 with F-Score: 0.363
permission  6
Length of sequence: 10000
Best Threshold: 0.0042 with F-Score: 0.5763
permission  7
Length of sequence: 10000
Best Threshold: 0.0025 with F-Score: 0.3529
permission  8
Length of sequence: 10000
Best Threshold: 0.0739 with F-Score: 0.4322
-------------------------------------
optimal threshold tuning for f-score
[0.2838, 0.0993, 0.0376, 0.0256, 0.0278, 0.267, 0.0042, 0.0025, 0.0739]


In [95]:
##for roc curve with g-mean

n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("acnet_predictions.npy")
  true_labels = np.load("acnet_labels.npy")

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Create the ROC curve
  fpr, tpr, thresholds = roc_curve(labels, pred)

  df_fpr_tpr = pd.DataFrame({'FPR':fpr, 'TPR':tpr, 'Threshold':thresholds})

  # Calculate the G-mean
  gmean = np.sqrt(tpr * (1 - fpr))

  # Find the optimal threshold
  index = np.argmax(gmean)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  gmeanOpt = round(gmean[index], ndigits = 4)
  fprOpt = round(fpr[index], ndigits = 4)
  tprOpt = round(tpr[index], ndigits = 4)

  thresh_roc[i] = thresholdOpt
  print('Best Threshold: {} with G-Mean: {}'.format(thresholdOpt, gmeanOpt))
  print('FPR: {}, TPR: {}'.format(fprOpt, tprOpt))

print("-------------------------------------")
print("ROC curve with G-mean threshold tuning")
print(thresh_roc)

permission  0
Best Threshold: 0.2011999934911728 with G-Mean: 0.7196
FPR: 0.2447, TPR: 0.6856
permission  1
Best Threshold: 0.04320000112056732 with G-Mean: 0.743
FPR: 0.2541, TPR: 0.7401
permission  2
Best Threshold: 0.007400000002235174 with G-Mean: 0.7855
FPR: 0.1991, TPR: 0.7703
permission  3
Best Threshold: 0.025599999353289604 with G-Mean: 0.7137
FPR: 0.2921, TPR: 0.7195
permission  4
Best Threshold: 0.6658999919891357 with G-Mean: 0.5963
FPR: 0.3763, TPR: 0.5702
permission  5
Best Threshold: 0.07329999655485153 with G-Mean: 0.7512
FPR: 0.2431, TPR: 0.7456
permission  6
Best Threshold: 0.00430000014603138 with G-Mean: 0.7887
FPR: 0.1381, TPR: 0.7217
permission  7
Best Threshold: 0.00019999999494757503 with G-Mean: 0.7855
FPR: 0.2102, TPR: 0.7812
permission  8
Best Threshold: 0.008799999952316284 with G-Mean: 0.792
FPR: 0.2908, TPR: 0.8846
-------------------------------------
ROC curve with G-mean threshold tuning
[0.2012, 0.0432, 0.0074, 0.0256, 0.6659, 0.0733, 0.0043, 0.0002, 0

#### d) Accuracy Score

In [ ]:
#Fscore micro for different thresholds-

In [96]:
#predictions = np.load("predictions.npy")
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [97]:
eval_accuracy = f_at_1(predictions, true_labels)

np.save("ACNET_F1_CV1_N20k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.
print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")
avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 score: {0:.4f}".format(avg_score))

  Camera    : 0.4708
  Location  : 0.5665
  Microphone: 0.5045
  Contacts  : 0.6054
  Storage   : 0.5941
  Phone     : 0.3630
  SMS       : 0.5763
  Call_Log  : 0.3529
  Calendar  : 0.4322

  Average F1 score: 0.4962


In [ ]:
#Fscore micro for different thresholds-

In [98]:
#predictions = np.load("predictions.npy")
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [99]:
eval_accuracy = f1micro_accuracy(predictions, true_labels)

np.save("ACNET_F1Mic_CV1_N20k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 (micro) score: {0:.4f}".format(avg_score))

  Camera    : 0.7694
  Location  : 0.7878
  Microphone: 0.8833
  Contacts  : 0.7115
  Storage   : 0.4752
  Phone     : 0.8685
  SMS       : 0.8409
  Call_Log  : 0.9144
  Calendar  : 0.9201

  Average F1 (micro) score: 0.7968


In [ ]:
#roc-auc score for different thresholds-

In [100]:
import numpy as np
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [101]:
#roc-auc score

eval_accuracy = roc_auc(predictions, true_labels)

np.save("ACNET_ROC_CV1_N20k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average ROC_AUC score: {0:.4f}".format(avg_score))

  Camera    : 0.7182
  Location  : 0.7430
  Microphone: 0.7844
  Contacts  : 0.7137
  Storage   : 0.5961
  Phone     : 0.7513
  SMS       : 0.7879
  Call_Log  : 0.7829
  Calendar  : 0.7969

  Average ROC_AUC score: 0.7416
